In [75]:
from enum import Enum
import yfinance as yf
import numpy as np

In [109]:
ticker = 'AAPL'

start_date = '2024-04-01'
end_date = '2024-04-02'

timeframe = '90m'
lookback = 30
deadline = 2

# Pips
take_profit = 30
stop_loss = 15

In [110]:
class PnL(Enum):
    TAKE_PROFIT = 0
    STOP_LOSS = 1
    NOTHING = 2

In [111]:
class PnLCalculator:
    def __init__(self, ticker, start_date, end_date, timeframe):
        # Add deadline to end_date so all days can be predicted
        self.data = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)
        # print(self.data)

    def get_pnl_labels(self, take_profit, stop_loss, deadline):
        res = []

        # Assume more data points than deadline
        for i in range(len(self.data['Close']) - deadline):
        # for i in range(len(self.data['Close'])):

            start_price = self.data['Close'].iloc[i]
            take_profit_price = start_price * (1 + take_profit / 1000)
            stop_loss_price = start_price * (1 - stop_loss / 1000)

            # sl_price, tp_price = compute(candles, type, data, func)
            # print(start_price, stop_loss_price, take_profit_price)

            label = PnL.NOTHING.value

            for j in range(i + 1, i + 1 + deadline):
            # for j in range(i + 1, min(i + 1 + deadline, len(self.data['Close']))):
                
                # Use high and low values
                price = self.data['Close'].iloc[j]

                if price >= take_profit_price:
                    label = PnL.TAKE_PROFIT.value
                    break
            
                if price <= stop_loss_price:
                    label = PnL.STOP_LOSS.value
                    break
                
            res.append(label)

        # Do we want array to be same size as data? What about end days where deadline longer than remaining days
        return np.array(res)

In [112]:
calculator = PnLCalculator(ticker, start_date, end_date, timeframe)
labels = calculator.get_pnl_labels(take_profit, stop_loss, deadline)
print(labels)
print(len(labels))

[*********************100%%**********************]  1 of 1 completed

                           Open        High         Low       Close  \
Datetime                                                              
2024-04-01 09:30:00  171.187500  171.229996  169.820007  169.880005   
2024-04-01 11:00:00  169.884995  170.080002  169.479996  169.895004   
2024-04-01 12:30:00  169.895004  170.089996  169.759995  169.996796   
2024-04-01 14:00:00  169.994995  170.240005  169.850006  169.934998   
2024-04-01 15:30:00  169.934998  170.139999  169.869995  170.009995   

                      Adj Close    Volume  
Datetime                                   
2024-04-01 09:30:00  169.880005  14668513  
2024-04-01 11:00:00  169.895004   7240330  
2024-04-01 12:30:00  169.996796   4276485  
2024-04-01 14:00:00  169.934998   4912380  
2024-04-01 15:30:00  170.009995   5786677  
169.8800048828125 167.33180480957031 174.97640502929687
169.89500427246094 167.346579208374 174.99185440063476
169.99679565429688 167.44684371948242 175.0966995239258
[2 2 2]
3
